In [1]:
# import libraries

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# turn off warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# set working directory

path = "S:/Kaggle Projects/kaggle-titanic"
os.chdir(path)

In [3]:
# read the dataset

data = pd.read_csv("train.csv")
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
# check for missing values

data.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

## Data Cleaning

In [5]:
# Replace missing values from`Age` with the median Age value

data['Age'] = data['Age'].fillna(data['Age'].median())

In [6]:
# Replace missing values of `Embarked` with the mode value

data['Embarked'] = data['Embarked'].fillna(data['Embarked'].mode()[0])

In [7]:
 # remove columns
    
data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [8]:
data.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [9]:
# Label encoding categorical columns

# import the module
from sklearn.preprocessing import LabelEncoder

# create object for encoder class
encoder = LabelEncoder()

In [10]:
# encode `Sex` column

data['Sex'] = encoder.fit_transform(data['Sex'])

In [11]:
# encode `Embarked` column

data['Embarked'] = encoder.fit_transform(data['Embarked'])

In [12]:
data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,1,0,3,1,22.0,1,0,7.2500,2
1,2,1,1,0,38.0,1,0,71.2833,0
2,3,1,3,0,26.0,0,0,7.9250,2
3,4,1,1,0,35.0,1,0,53.1000,2
4,5,0,3,1,35.0,0,0,8.0500,2


## Train classifier model

In [37]:
# import modules

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

In [14]:
pipe_svc = make_pipeline(StandardScaler(), 
                         VarianceThreshold(threshold=0),
                         LinearSVC(random_state=98))

pipe_lr = make_pipeline(StandardScaler(), 
                        VarianceThreshold(threshold=0),
                        LogisticRegression(random_state=89))

In [15]:
# define parameters to tune

params_svc = {
    'linearsvc__C': [0.1, 0.5, 1, 10],
    'linearsvc__loss': ['hinge', 'squared_hinge']
}

params_lr = {
    'logisticregression__penalty': ['l1', 'l2'],
    'logisticregression__C': [0.1, 0.5, 1, 10],
    'logisticregression__solver': ['liblinear', 'saga']
}
    

In [16]:
# split data into train and test data

X = data.iloc[:, [0, 2, 3, 4, 5, 6, 7, 8]]
y = data.iloc[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=90, stratify=y)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 8) (179, 8) (712,) (179,)


In [44]:
# apply gridSearchCV to data

svc_grid_cv = GridSearchCV(estimator=pipe_svc,
                         param_grid=params_svc,
                         scoring='accuracy',
                         cv=10)

lr_grid_cv = GridSearchCV(estimator=pipe_lr,
                         param_grid=params_lr,
                         scoring='accuracy',
                         cv=10)

# combine the 2 grid cv into a list
grid_cv = [svc_grid_cv, lr_grid_cv]

In [45]:
for model in grid_cv:
    model.fit(X_train, y_train)

In [19]:
model_dict = {0: 'SVC', 1: 'Logistic regression'}

for i, model in enumerate(grid_cv):
    print(f"{model_dict[i]} Accuracy: {model.score(X_test, y_test)}")
    print(f"{model_dict[i]} Optimal parameters: \n {model.best_params_}")

SVC Accuracy: 0.8379888268156425
SVC Optimal parameters: 
 {'linearsvc__C': 0.5, 'linearsvc__loss': 'hinge'}
Logistic regression Accuracy: 0.8324022346368715
Logistic regression Optimal parameters: 
 {'logisticregression__C': 0.1, 'logisticregression__penalty': 'l1', 'logisticregression__solver': 'saga'}


In [21]:
svm_clf = LinearSVC(C=0.5, loss="hinge", random_state=90)
svm_clf.fit(X_train, y_train)

LinearSVC(C=0.5, loss='hinge', random_state=90)

In [23]:
y_pred = svm_clf.predict(X_test)

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix

print(accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

0.8435754189944135
[[101   9]
 [ 19  50]]


## Test data

### Data Cleaning

In [25]:
# import the test data

test_data = pd.read_csv("test.csv")

In [26]:
# check for missing values

test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [27]:
# replace missing values of Age and Fare with median

test_data['Age'] = test_data['Age'].fillna(test_data['Age'].median())

test_data['Fare'] = test_data['Fare'].fillna(test_data['Fare'].median())

In [28]:
# remove columns 

test_data.drop(['Name', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [29]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,male,34.5,0,0,7.8292,Q
1,893,3,female,47.0,1,0,7.0000,S
2,894,2,male,62.0,0,0,9.6875,Q
3,895,3,male,27.0,0,0,8.6625,S
4,896,3,female,22.0,1,1,12.2875,S


In [30]:
# encode the text columns: `Sex` and `Embarked`

test_data['Sex'] = encoder.fit_transform(test_data['Sex'])

test_data['Embarked'] = encoder.fit_transform(test_data['Embarked'])

In [31]:
test_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,892,3,1,34.5,0,0,7.8292,1
1,893,3,0,47.0,1,0,7.0000,2
2,894,2,1,62.0,0,0,9.6875,1
3,895,3,1,27.0,0,0,8.6625,2
4,896,3,0,22.0,1,1,12.2875,2


## Test the model

In [32]:
y_pred = svm_clf.predict(test_data)

In [34]:
prediction_data = {'PassengerId': test_data['PassengerId'],
                  'Survived': y_pred}

In [35]:
final_df = pd.DataFrame(prediction_data)

In [36]:
final_df.to_csv('submission.csv',
               encoding='utf-8',
               index=False)